In [9]:
import gensim
import tensorflow as tf

from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, MultiHeadAttention, Dropout, LayerNormalization, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model


In [7]:
# gensim으로 Word2Vec 모델 로드
word2vec_model = gensim.models.Word2Vec.load('ko.bin')

word2vec_model.wv.most_similar("사람")

[('젊은이', 0.6494427919387817),
 ('여인', 0.6287257671356201),
 ('백성', 0.6063710451126099),
 ('포졸', 0.6043275594711304),
 ('죄인', 0.5960500836372375),
 ('선비', 0.5868039131164551),
 ('부녀자', 0.5654411315917969),
 ('죄수', 0.5639811754226685),
 ('구경꾼', 0.5620019435882568),
 ('손님', 0.5589558482170105)]

In [8]:
type(word2vec_model)

gensim.models.word2vec.Word2Vec

In [ ]:
# 임베딩 벡터의 크기와 어휘 크기를 가져옵니다.
embedding_matrix = word2vec_model.vectors
embedding_dim = word2vec_model.vector_size
vocab_size = len(word2vec_model.key_to_index) + 1  # 0 인덱스를 위해 1을 더합니다.

print('embedding dimension: ', embedding_dim)
print('vocab_size: ', vocab_size)

In [10]:
# 양방향 LSTM 모델

# 임베딩 차원과 어휘 사전의 크기는 임의로 설정
# 실제 적용 시 데이터에 맞게 조정 필요
embedding_dim = 128
vocab_size = 10000  # 어휘 사전의 크기 (임의로 설정한 값)

# 모델의 입력 정의: 이 예제에서는 임의의 길이의 시퀀스를 처리할 수 있도록 None을 사용
sequence_input = Input(shape=(None,), dtype='int32')

# 임베딩 레이어
embedded_sequences = Embedding(vocab_size, embedding_dim)(sequence_input)

# 양방향 LSTM 레이어 2개를 쌓음
x = Bidirectional(LSTM(64, return_sequences=True))(embedded_sequences)  # 첫 번째 LSTM
x = Bidirectional(LSTM(64))(x)  # 두 번째 LSTM

# Dense 레이어
x = Dense(64, activation='relu')(x)

# 출력 레이어: 4개의 클래스에 대한 예측을 출력
predictions = Dense(4, activation='softmax')(x)

# 최종 모델 생성
model_bilstm = Model(sequence_input, predictions)

# 모델 컴파일
model_bilstm.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 구조 요약 출력
model_bilstm.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 1,486,148
Trainable params: 1,486,148
Non-trainable params: 0
___________________________________________________

In [11]:
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, Dropout, LayerNormalization, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

# 임베딩 차원과 어휘 사전의 크기, 시퀀스의 최대 길이는 임의로 설정
# 실제 적용 시 데이터에 맞게 조정 필요
embedding_dim = 128
vocab_size = 10000  # 어휘 사전의 크기 (임의로 설정한 값)
max_seq_length = 512  # 시퀀스의 최대 길이 (임의로 설정한 값)
num_heads = 4  # 멀티 헤드 어텐션에서의 헤드 수 (임의로 설정한 값)
ff_dim = 128  # 피드포워드 네트워크의 차원 수 (임의로 설정한 값)

# 모델의 입력 정의
sequence_input = Input(shape=(max_seq_length,), dtype='int32')

# 임베딩 레이어
embedded_sequences = Embedding(vocab_size, embedding_dim)(sequence_input)

# 트랜스포머 블록
# 멀티 헤드 어텐션
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(embedded_sequences, embedded_sequences)
attention_output = Dropout(0.1)(attention_output)
attention_output = LayerNormalization(epsilon=1e-6)(embedded_sequences + attention_output)

# 피드포워드 네트워크
ffn_output = Dense(ff_dim, activation='relu')(attention_output)
ffn_output = Dropout(0.1)(ffn_output)
sequence_output = LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)

# 글로벌 평균 풀링을 사용하여 시퀀스 차원을 축소
pooled_output = GlobalAveragePooling1D()(sequence_output)

# Dense 레이어
x = Dense(128, activation='relu')(pooled_output)

# 출력 레이어: 4개의 클래스에 대한 예측을 출력
predictions = Dense(4, activation='softmax')(x)

# 최종 모델 생성
model_tr = Model(sequence_input, predictions)

# 모델 컴파일
model_tr.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 구조 요약 출력
model_tr.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 512, 128)     1280000     input_2[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 512, 128)     263808      embedding_1[0][0]                
                                                                 embedding_1[0][0]                
__________________________________________________________________________________________________
dropout (Dropout)               (None, 512, 128)     0           multi_head_attention[0][0] 